In [7]:
!pip install -U --no-cache-dir \
  transformers==4.44.2 \
  tokenizers==0.19.1 \
  accelerate==0.33.0 \
  datasets==3.0.1  \
  scikit-learn==1.4.2

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 9.8 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.2
    Uninstalling scikit-learn-1.7.2:
      Successfully uninstalled scikit-learn-1.7.2
  Attempting uninstall: multiprocess━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [scikit-learn]
    Found existing installation: multiprocess 0.70.18━━━━━━━━━ 1/5 [scikit-learn]
    Uninstalling multiprocess-0.70.18:━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [scikit-learn]
      Successfully uninstalled multiprocess-0.70.18━━━━━━━━━━━ 1/5 [scikit-learn]
  Attempting uninstall: accelerate━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [scikit-learn]
    Found existing installatio

In [8]:
from datasets import load_dataset
ds = load_dataset("jeanlee/kmhas_korean_hate_speech")

README.md: 0.00B [00:00, ?B/s]

kmhas_korean_hate_speech.py: 0.00B [00:00, ?B/s]

Using the latest cached version of the dataset since jeanlee/kmhas_korean_hate_speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/c657d15baf277c48d467f0625f7d33c50d4352ef (last modified on Sat Dec 13 17:39:23 2025).


In [17]:
# load train, validation, and test dataset from HuggingFace

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")


# 🔽 원하는 크기로 줄이기
train_n = 15000
val_n   = 1500
test_n  = 3000

train = train.shuffle(seed=42).select(range(min(train_n, len(train))))
validation = validation.shuffle(seed=42).select(range(min(val_n, len(validation))))
test = test.shuffle(seed=42).select(range(min(test_n, len(test))))


Using the latest cached version of the dataset since jeanlee/kmhas_korean_hate_speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/c657d15baf277c48d467f0625f7d33c50d4352ef (last modified on Sun Dec 14 00:57:37 2025).
Using the latest cached version of the dataset since jeanlee/kmhas_korean_hate_speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/c657d15baf277c48d467f0625f7d33c50d4352ef (last modified on Sun Dec 14 00:57:37 2025).
Using the latest cached version of the dataset since jeanlee/kmhas_korean_hate_speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/c657d15baf277c48d4

In [19]:
def to_binary(example):
    labels = example["label"]
    example["y"] = 0 if labels ==[8] else 1
    return example

ds_bin = ds.map(to_binary)

In [20]:
import re
PATTERN = re.compile(
 r"(ㅅㅂ|ㅂㅅ|ㅈㄴ|씨발|병신|개새|좆|지랄|미친|틀딱|맘충|김치녀|한남)"
)


def baseline_predict(text: str) -> int:
    return 1 if PATTERN.search(text) else 0

In [56]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

dev = ds_bin["validation"]

y_true = dev["y"]
y_pred_baseline = [baseline_predict(t) for t in dev["text"]]


acc_b = accuracy_score(y_true, y_pred_baseline)

f1_macro_b = f1_score(y_true, y_pred_baseline, average="macro")
f1_micro_b = f1_score(y_true, y_pred_baseline, average="micro")
f1_weighted_b =  f1_score(y_true, y_pred_baseline, average="weighted")
precision_macro_b =  precision_score(y_true, y_pred_baseline, average = "macro", zero_division= 0)
recall_macro_b= recall_score(y_true, y_pred_baseline, average = "macro", zero_division = 0)






In [22]:
models = [
    "monologg/koelectra-base-v3-discriminator"
]



def train_only(
    model_name,
    ds_bin,
    out_dir,
    max_length,
    batch_size,
    epochs,
    lr,
    fp16=True
):
    from transformers import (
        AutoTokenizer,
        AutoModelForSequenceClassification,
        Trainer,
        TrainingArguments
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=2
    )

    def tokenize(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )

    ds_tok = ds_bin.map(tokenize, batched=True)
    ds_tok = ds_tok.rename_column("y", "labels")
    ds_tok.set_format("torch", columns=["input_ids","attention_mask","labels"])

    args = TrainingArguments(
        output_dir=out_dir,
        do_train=True,
        do_eval=False,                 # 🔴 eval 안 함
        evaluation_strategy="no",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=fp16,
        logging_steps=100,
        report_to="none",
        save_safetensors=False,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        tokenizer=tokenizer,
    )

    trainer.train()
    trainer.save_model(out_dir)

    tokenizer.save_pretrained(out_dir)




In [24]:
import os

results = []

for m in models:
    if "large" in m.lower():
        bs, ml = 2, 96
    else:
        bs, ml = 16, 64

    out_dir = (
        f"./runs/"
        f"{m.replace('/','_')}"
        f"_train-ds-small"
        f"_ep1"
        f"_lr2e-5"
    )

    # ---------- TRAIN ----------
    if not os.path.exists(out_dir):
        train_only(
            model_name=m,
            ds_bin=ds_bin,
            out_dir=out_dir,
            max_length=ml,
            batch_size=bs,
            epochs=1,
            lr=2e-5,
        )


#pd.DataFrame(results).sort_values("test_f1_macro", ascending=False)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
100,0.619900
200,0.481800
300,0.420000
400,0.393500
500,0.387400
600,0.369700
700,0.367400
800,0.379300
900,0.329500
1000,0.372800


In [38]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
  #  probs = 1 / (1 + np.exp(-logits)) if logits.shape[1] == 1 else None

    preds = np.argmax(logits, axis=-1)

    out = {
        "acc": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
        "precision_macro": precision_score(labels, preds, average = "macro", zero_division= 0),
        "recall_macro": recall_score(labels, preds, average = "macro", zero_division = 0)
    }
    return out


In [39]:
def eval_only(model_dir, ds_bin, split, max_length, batch_size):
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)

    def tokenize(batch):
        return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_length)

    ds_split = ds_bin[split].map(tokenize, batched=True)
    ds_split = ds_split.rename_column("y", "labels")
    ds_split.set_format("torch", columns=["input_ids","attention_mask","labels"])

    args = TrainingArguments(
        output_dir="./eval_tmp",
        per_device_eval_batch_size=batch_size,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=args,
        eval_dataset=ds_split,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    return trainer.evaluate()


In [40]:
results = []

for m in models:
    if "large" in m.lower():
        bs, ml = 2, 96
    else:
        bs, ml = 16, 64

    out_dir = (
        f"./runs/"
        f"{m.replace('/','_')}"
        f"_train-ds-small"
        f"_ep1"
        f"_lr2e-5"
    )

    # ---------- EVAL ----------
    val_metrics = eval_only(out_dir, ds_bin, "validation", ml, bs)
    test_metrics = eval_only(out_dir, ds_bin, "test", ml, bs)




Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [42]:
#results
results.append({
    "model": m,
    "out_dir": out_dir,

    # ---- F1 scores ----
    "val_f1_macro": val_metrics["eval_f1_macro"],
    "test_f1_macro": test_metrics["eval_f1_macro"],

    "val_f1_micro": val_metrics["eval_f1_micro"],
    "test_f1_micro": test_metrics["eval_f1_micro"],

    "val_f1_weighted": val_metrics["eval_f1_weighted"],
    "test_f1_weighted": test_metrics["eval_f1_weighted"],

    # ---- Accuracy ----
    "val_acc": val_metrics["eval_acc"],
    "test_acc": test_metrics["eval_acc"],

    # ---- Precision / Recall (macro) ----
    "val_precision_macro": val_metrics.get("eval_precision_macro"),
    "test_precision_macro": test_metrics.get("eval_precision_macro"),

    "val_recall_macro": val_metrics.get("eval_recall_macro"),
    "test_recall_macro": test_metrics.get("eval_recall_macro"),
})




,model,out_dir,val_f1_macro,test_f1_macro,val_f1_micro,val_f1_weighted,test_f1_weighted,val_acc,test_acc,test_f1_micro,val_precision_macro,test_precision_macro,val_recall_macro,test_recall_macro
0,monologg/koelectra-base-v3-discriminator,./runs/monologg_koelectra-base-v3-discriminato...,0.885286,0.888874,0.887078,0.886917,0.886917,0.887078,0.887078,NaN,NaN,NaN,NaN,NaN
1,monologg/koelectra-base-v3-discriminator,./runs/monologg_koelectra-base-v3-discriminato...,0.885286,0.888874,0.887078,0.886917,0.889549,0.887078,0.889557,0.889557,0.886508,0.888934,0.884299,0.888816


In [57]:
results.append({
    "model": "Baseline (keyword)",
    "out_dir": "N/A",

    "val_f1_macro": None,
    "test_f1_macro": f1_macro_b,

    "val_f1_micro": None,
    "test_f1_micro": f1_micro_b,

    "val_f1_weighted": None,
    "test_f1_weighted": f1_weighted_b,

    "val_acc": None,
    "test_acc": acc_b,

    "val_precision_macro": None,
    "test_precision_macro": precision_macro_b,

    "val_recall_macro": None,
    "test_recall_macro": recall_macro_b,
})


In [58]:

import pandas as pd
df = pd.DataFrame(results)
df = df.round(4)
df.to_csv("results.csv", index = False)

In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

def get_model_preds(model_dir, ds_bin, split, max_length=64, batch_size=16):
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)

    def tokenize(batch):
        return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_length)

    ds_split = ds_bin[split].map(tokenize, batched=True)
    ds_split = ds_split.rename_column("y", "labels")
    ds_split.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    args = TrainingArguments(
        output_dir="./pred_tmp",
        per_device_eval_batch_size=batch_size,
        report_to="none",
    )

    trainer = Trainer(model=model, args=args, tokenizer=tokenizer)

    pred_out = trainer.predict(ds_split)
    logits = pred_out.predictions
    preds = np.argmax(logits, axis=-1)  # ✅ 너가 물어본 그 argmax가 여기서 씀

    # (옵션) class=1 확률도 같이 뽑기 (softmax)
    exps = np.exp(logits - logits.max(axis=1, keepdims=True))
    probs = exps / exps.sum(axis=1, keepdims=True)
    prob_hate = probs[:, 1]  # label=1 확률

    return preds, prob_hate

def make_compare_df(ds_bin, split, model_dir, max_length=64, batch_size=16):
    texts = ds_bin[split]["text"]
    gold  = np.array(ds_bin[split]["y"])

    baseline = np.array([baseline_predict(t) for t in texts])

    model_pred, model_prob = get_model_preds(
        model_dir=model_dir,
        ds_bin=ds_bin,
        split=split,
        max_length=max_length,
        batch_size=batch_size
    )

    df = pd.DataFrame({
        "text": texts,
        "gold": gold,
        "baseline": baseline,
        "model": model_pred,
        "model_prob_hate": model_prob
    })

    return df

df_test = make_compare_df(
    ds_bin=ds_bin,
    split="test",
    model_dir="./runs/monologg_koelectra-base-v3-discriminator_train-ds-small_ep1_lr2e-5",
    max_length=64,
    batch_size=16
)

diff_bm = df_test[df_test["baseline"] != df_test["model"]].copy()
diff_bm.sort_values("model_prob_hate", ascending=False).head(10)


,model,out_dir,val_f1_macro,test_f1_macro,val_f1_micro,val_f1_weighted,test_f1_weighted,val_acc,test_acc,test_f1_micro,val_precision_macro,test_precision_macro,val_recall_macro,test_recall_macro
0,monologg/koelectra-base-v3-discriminator,./runs/monologg_koelectra-base-v3-discriminato...,0.8853,0.8889,0.8871,0.8869,0.8869,0.8871,0.8871,NaN,NaN,NaN,NaN,NaN
1,monologg/koelectra-base-v3-discriminator,./runs/monologg_koelectra-base-v3-discriminato...,0.8853,0.8889,0.8871,0.8869,0.8895,0.8871,0.8896,0.8896,0.8865,0.888934,0.8843,0.8888
2,Baseline (keyword),N/A,NaN,0.4773,NaN,NaN,0.5069,NaN,0.6067,0.6067,NaN,"(0.7531514082664958,)",NaN,0.5573
3,Baseline (keyword),N/A,NaN,0.4773,NaN,NaN,0.5069,NaN,0.6067,0.6067,NaN,"(0.7531514082664958,)",NaN,0.5573
4,Baseline (keyword),N/A,NaN,0.4773,NaN,NaN,0.5069,NaN,0.6067,0.6067,NaN,0.753151,NaN,0.5573


In [ ]:
diff_gm = df_test[df_test["gold"] != df_test["model"]].copy()
diff_gm.sort_values("model_prob_hate", ascending=False).head(10)


In [ ]:
sample3 = diff_bm.sample(3, random_state=42)
sample3[["text", "gold", "baseline", "model", "model_prob_hate"]]
